In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import os
print(os.getcwd()) 

X_train = pd.read_csv("featureset/X_train_features.csv", index_col=0)
X_test = pd.read_csv("featureset/X_test_features.csv", index_col=0)
y_train = pd.read_csv('Clean Data/y_train_prep.csv', index_col=0)
y_test = pd.read_csv('Clean Data/y_test_prep.csv', index_col=0)



C:\Users\matze\Documents\Winton


In [147]:
weights_daily = pd.read_csv('Clean Data/weight_daily.csv', index_col=0, header = None).loc[X_test.index,:].values.reshape(-1,1).flatten()

intraday_weights = pd.read_csv('Clean Data/weight_intraday.csv', index_col=0, header = None).loc[X_test.index,:].values.reshape(-1,1).flatten()
weights_daily.shape

(13200,)

### Interaction

In [129]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(interaction_only=True)
Train_Poly = poly.fit_transform(X_train)
Test_Poly = poly.fit_transform(X_test)

# Scorer

In [4]:
from sklearn.metrics import mean_squared_error

In [5]:
train_zeros = pd.DataFrame(np.zeros(y_train.shape), columns = y_train.columns)
test_zeros = pd.DataFrame(np.zeros(y_test.shape), columns = y_test.columns)

In [16]:
test_score = mean_squared_error(y_test, test_zeros)
test_score_minutes = mean_squared_error(y_test.loc[:,'Ret_121':'Ret_180'], test_zeros.loc[:,'Ret_121':'Ret_180'])
test_score_daily = mean_squared_error(y_test.loc[:,'Ret_PlusOne':'Ret_PlusTwo'], test_zeros.loc[:,'Ret_PlusOne':'Ret_PlusTwo'])

In [18]:
train_score = mean_squared_error(y_train, train_zeros)
train_score_minutes = mean_squared_error(y_train.loc[:,'Ret_121':'Ret_180'], train_zeros.loc[:,'Ret_121':'Ret_180'])
train_score_daily = mean_squared_error(y_train.loc[:,'Ret_PlusOne':'Ret_PlusTwo'], train_zeros.loc[:,'Ret_PlusOne':'Ret_PlusTwo'])

# Regression Models

### Random Forest

In [46]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(random_state=0)
regr.fit(X_train, y_train.Ret_PlusOne)
pred = regr.predict(X_test)
print('RF 10 Ret_PlusOne ')
print(' Score',mean_squared_error(pred, y_test.Ret_PlusOne))
print('Zero: ', test_score_daily)

RF 10 Ret_PlusOne 
 Score 0.0007020410438811058
Zero:  0.0005966937810296722


In [47]:
regr100 = RandomForestRegressor(n_estimators =50 ,random_state=0)
regr100.fit(X_train, y_train.Ret_PlusOne)
pred100 = regr100.predict(X_test)
mean_squared_error(pred100, y_test.Ret_PlusOne)

0.0005996651960335922

In [21]:
test_score_daily

0.0005966937810296722

In [48]:
regr1 = RandomForestRegressor(n_estimators =1 ,random_state=0)
regr1.fit(X_train, y_train.Ret_PlusOne)
pred1 = regr1.predict(X_test)
print('RF 1 Ret_PlusOne ')
print(' Score',mean_squared_error(pred1, y_test.Ret_PlusOne))
print('Zero: ', test_score_daily)

RF 1 Ret_PlusOne 
 Score 0.0019056509496557918
Zero:  1.3923236682626972e-06


In [35]:
regr1 = RandomForestRegressor(n_estimators =1 ,random_state=0)
regr1.fit(X_train, y_train.Ret_121)
pred1 = regr1.predict(X_test)
print('RF 1 Ret_121 ')
print(' Score',mean_squared_error(pred1, y_test.Ret_121))
print('Zero: ', test_score_minutes)

RF 1 Ret_121 
 Score 1.2474362135587695e-06
Zero:  1.3923236682626972e-06


In [49]:
regr1 = RandomForestRegressor(n_estimators =10 ,random_state=0)
regr1.fit(X_train, y_train.Ret_121)
pred1 = regr1.predict(X_test)
print('RF 10 Ret_121 ')
print(' Score',mean_squared_error(pred1, y_test.Ret_121))
print('Zero: ', test_score_minutes)

RF 10 Ret_121 
 Score 1.2441529253613372e-06
Zero:  1.3923236682626972e-06


In [52]:
pred1 = regr1.predict(X_train)
print('RF 10 Ret_PlusOne ')
print(' Score',mean_squared_error(pred1, y_train.Ret_121))
print('Zero: ', train_score_minutes)
print('Zero: ', test_score_minutes)

RF 10 Ret_PlusOne 
 Score 2.204865700239446e-07
Zero:  1.344453902618752e-06
Zero:  1.3923236682626972e-06


In [53]:
regr1 = RandomForestRegressor(n_estimators =100 ,random_state=0)
regr1.fit(X_train, y_train.Ret_121)
pred1 = regr1.predict(X_test)
print('RF 10 Ret_121 ')
print(' Score',mean_squared_error(pred1, y_test.Ret_121))
print('Zero: ', test_score_minutes)

RF 10 Ret_121 
 Score 1.1075681188009348e-06
Zero:  1.3923236682626972e-06


(13200,)

### Ridge/Lasso

In [171]:
from sklearn import linear_model
lasso01 = linear_model.Lasso(alpha=0.1, normalize = False)
lasso01.fit(X_train,y_train.Ret_121)
pred_lasso01 = lasso01.predict(X_test)
print('Lasso 01 Ret_121 ')
print('Score',mean_squared_error(pred_lasso01, y_test.Ret_121, sample_weight = intraday_weights))
print('Zero: ', test_score_minutes)

Lasso 01 Ret_121 
Score 1.0736013733432277e-06
Zero:  1.3923236682626972e-06


In [169]:
from sklearn import linear_model
lasso01 = linear_model.Lasso(alpha=0.1)
lasso01.fit(X_train,y_train.Ret_122)
pred_lasso01 = lasso01.predict(X_test)
print('Lasso 01 Ret_122 ')
print('Score',mean_squared_error(pred_lasso01, y_test.Ret_122, sample_weight = intraday_weights))
print('Zero: ', test_score_minutes)

Lasso 01 Ret_122 
Score 1.2341529143992744e-06
Zero:  1.3923236682626972e-06


In [168]:
from sklearn import linear_model
lasso01 = linear_model.Lasso(alpha=0.1, normalize = True)
lasso01.fit(X_train,y_train.loc[:,'Ret_121':'Ret_129'])
pred_lasso01 = lasso01.predict(X_test)
print('Lasso 01 Ret_123 ')
print('Score',mean_squared_error(pred_lasso01, y_test.loc[:,'Ret_121':'Ret_129']))
print('Zero: ', test_score_minutes)

Lasso 01 Ret_123 
Score 1.2292176985223422e-06
Zero:  1.3923236682626972e-06


In [172]:
from sklearn import linear_model
lasso01 = linear_model.Lasso(alpha=0.1)
lasso01.fit(X_train,y_train.Ret_PlusOne)
pred_lasso01 = lasso01.predict(X_test)
print('Lasso 01 Ret_123 ')
print('Score',mean_squared_error(pred_lasso01, y_test.Ret_PlusOne, sample_weight = weights_daily))
print('Zero:', test_score_daily)


Lasso 01 Ret_123 
Score 0.0005354886744994992
Zero: 0.0005966937810296722


Poly Features

In [167]:
lasso01 = linear_model.Lasso(alpha=0.1, normalize = True)
lasso01.fit(Train_Poly, y_train.Ret_PlusOne)
pred_lasso01 = lasso01.predict(Test_Poly)
print('Lasso 01 Ret_123 ')
print('Score',mean_squared_error(pred_lasso01, y_test.Ret_PlusOne, sample_weight = weights_daily))
print('Zero: ', test_score_daily)


Lasso 01 Ret_123 
Score 0.0005354518924535067
Zero:  0.0005966937810296722


### GBR

In [54]:
from sklearn.ensemble import GradientBoostingRegressor

In [160]:
gbr = GradientBoostingRegressor(random_state = 0)
gbr.fit(X_train, y_train.Ret_121)
pred_gbr = gbr.predict(X_test)
print('GBR 100 Ret_121 ')
print('Score',mean_squared_error(pred_gbr, y_test.Ret_121, sample_weight = weights_daily))
print('Zero: ', test_score_minutes)

GBR 100 Ret_121 
Score 1.0157865930115265e-06
Zero:  1.3923236682626972e-06


In [ ]:
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train.Ret_121)
pred_gbr = gbr.predict(X_test)
print('GBR 100 Ret_121 ')
print('Score',mean_squared_error(pred_gbr, y_test.Ret_121))
print('Zero: ', test_score_minutes)

In [112]:
gbr = GradientBoostingRegressor(n_estimators= 100, random_state=0)
gbr.fit(X_train, y_train.Ret_PlusOne)
pred_gbr = gbr.predict(X_test)
print('GBR 100 Ret_PlusOne ')
print('Score',mean_squared_error(pred_gbr, y_test.Ret_PlusOne))
print('Zero: ', test_score_daily)

GBR 100 Ret_PlusOne 


TypeError: 1D weights expected when shapes of a and weights differ.

# Generating Output